# Dataset Statistics for SynthDet
This example notebook shows how to use datasetinsights to load synthetic datasets generated from the [SynthDet](https://github.com/Unity-Technologies/synthdet) example project and visualize dataset statistics.

In addition to the object bounding boxes, SynthDet produces a mix of built-in and project-specific metrics. Statistics for the `RenderedObjectInfo` metrics built into the Perception package can be calculated using `datasetinsights.stats.statistics.RenderedObjectInfo`. SynthDet-specific statistics are loaded via `datasetinsights.datasets.unity_perception.Metrics` and are calculated directly in this notebook.

## Setup dataset
If the dataset was generated locally, point `data_root` below to the path of the dataset. If you haven't generated or downloaded dataset locally, you can download it locally inside data_root using Unity Simulation or SynthDet Data Download cells below.

In [ ]:
data_root = "/data"

### Unity Simulation [Optional]
If the dataset was generated on Unity Simulation, the following cells can be used to download the metrics needed for dataset statistics.

Provide the `run-execution-id` which generated the dataset and a valid `access_token` in the following cell. The `access_token` can be generated using the Unity Simulation [CLI](https://github.com/Unity-Technologies/Unity-Simulation-Docs/blob/master/doc/cli.md#usim-inspect-auth).

In [ ]:
from datasetinsights.io import create_downloader
#run execution id:
# run_execution_id = "xxx"
# #access_token:
# access_token = "xxx"
# #annotation definition id:
# annotation_definition_id = "6716c783-1c0e-44ae-b1b5-7f068454b66e"
# #unity project id
# project_id = "xxx"
# usim_source_uri = f"usim://{project_id}/{run_execution_id}"

# unity_simulation_downloader = create_downloader(source_uri=usim_source_uri, access_token=access_token)


Before loading the dataset metadata for statistics we first download the relevant files from Unity Simulation. 

In [ ]:
unity_simulation_downloader.download(source_uri=usim_source_uri, output=data_root)

### SynthDet Data Download [Optional]
If you haven't used USim to donwload data, alternatively you can download public synthdet dataset which consists of around 5k synthetic images.

In [ ]:
# http_source_uri="https://storage.googleapis.com/datasetinsights/data/groceries/v3.zip" 
# http_downloader = create_downloader(source_uri=http_source_uri)

# http_downloader.download(source_uri=source_uri,
#                     output=data_root)

## Load dataset metadata
Once the dataset metadata is downloaded, it can be loaded for statistics using `datasetinsights.datasets.unity_perception`. Annotation and metric definitions are loaded into pandas dataframes using `AnnotationDefinitions` and `MetricDefinitions` respectively.

In [ ]:
from datasetinsights.datasets.unity_perception import AnnotationDefinitions, MetricDefinitions
ann_def = AnnotationDefinitions(data_root)
ann_def.table

In [ ]:
metric_def = MetricDefinitions(data_root)
metric_def.table

## Built-in Statistics
The following tables and charts are supplied by `datasetinsights.stats.statistics.RenderedObjectInfo` on datasets that include the "rendered object info" metric.

In [ ]:
import datasetinsights.stats.statistics as stat
from datasetinsights.stats import bar_plot, histogram_plot, rotation_plot, plot_bboxes

max_samples = 10000          # maximum number of samples points used in histogram plots

rendered_object_info_definition_id = "659c6e36-f9f8-4dd6-9651-4a80e51eabc4"
roinfo = stat.RenderedObjectInfo(data_root=data_root, def_id=rendered_object_info_definition_id)

### Descriptive Statistics

In [ ]:
roinfo.num_captures()
roinfo.raw_table.head(3)

### Total Object Count

In [ ]:
total_count = roinfo.total_counts()
total_count

In [ ]:
bar_plot(
    total_count, 
    x="label_id", 
    y="count", 
    x_title="Label Name",
    y_title="Count",
    title="Total Object Count in Dataset",
    hover_name="label_name"
)

### Per Capture Object Count

In [ ]:
per_capture_count = roinfo.per_capture_counts()
per_capture_count.head(10)

In [ ]:
histogram_plot(
    per_capture_count, 
    x="count",  
    x_title="Object Counts Per Capture",
    y_title="Frequency",
    title="Distribution of Object Counts Per Capture",
    max_samples=max_samples
)

### Object Visible Pixels

In [ ]:
histogram_plot(
    roinfo.raw_table, 
    x="visible_pixels",  
    x_title="Visible Pixels Per Object",
    y_title="Frequency",
    title="Distribution of Visible Pixels Per Object",
    max_samples=max_samples
)

## SynthDet Statistics
Metrics specific to the simulation can be loaded using `datasetinsights.datasets.unity_perception.Metrics`. 

In [ ]:
from datasetinsights.datasets.unity_perception import Metrics
metrics = Metrics(data_root=data_root)

### Foreground placement info

In [ ]:
import pandas as pd

foreground_placement_info_definition_id = "061e08cc-4428-4926-9933-a6732524b52b"
columns = ("x_rot", "y_rot", "z_rot")

def read_foreground_placement_info(metrics):
    filtered_metrics = metrics.filter_metrics(foreground_placement_info_definition_id)
    combined = pd.DataFrame(filtered_metrics["rotation"].to_list(), columns=columns)
    
    return combined

In [ ]:
orientation = read_foreground_placement_info(metrics)
orientation.head(10)

In [ ]:
rotation_plot(
    orientation,
    x="x_rot",
    y="y_rot",
    z="z_rot",
    title="Object orientations",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    orientation, 
    x="x_rot",  
    x_title="Object Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Object Rotations along X direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    orientation, 
    x="y_rot",  
    x_title="Object Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Object Rotations along Y direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    orientation, 
    x="z_rot",  
    x_title="Object Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Object Rotations along Z direction",
    max_samples=max_samples
)

### Lighting info 

In [ ]:
lighting_info_definition_id = "939248ee-668a-4e98-8e79-e7909f034a47"
x_y_columns = ["x_rotation", "y_rotation"]
color_columns = ["color.r", "color.g", "color.b", "color.a"]

def read_lighting_info(metrics):
    filtered_metrics = metrics.filter_metrics(lighting_info_definition_id)
    colors = pd.json_normalize(filtered_metrics["color"])
    colors.columns = color_columns
    combined = pd.concat([filtered_metrics[x_y_columns], colors], axis=1, join='inner')

    return combined

In [ ]:
lighting = read_lighting_info(metrics)
lighting.head(5)

In [ ]:
rotation_plot(
    lighting,
    x="x_rotation",
    y="y_rotation",
    title="Light orientations",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting, 
    x="x_rotation",  
    x_title="Lighting Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Lighting Rotations along X direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting, 
    x="y_rotation",  
    x_title="Lighting Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Lighting Rotations along Y direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting, 
    x="color.r",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Redness",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting, 
    x="color.g",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Greeness",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting, 
    x="color.b",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Blueness",
    max_samples=max_samples
)

## Images With 2D Bounding Boxes

In this section, we provide sample code to render 2d bounding boxes on top of the captured images.

### Unity Simulation [Optional]
If the dataset was generated on Unity Simulation, the following cells can be used to download the images, captures and annotations in the dataset. Make sure you have enough disk space to store all files. For example, a dataset with 100K captures requires roughly 300GiB storage.

In [ ]:
#unity_simulation_downloader.download(source_uri=usim_source_uri, output=data_root, include_binary=True)

### Captures

In [ ]:
from datasetinsights.datasets.unity_perception import Captures
bounding_box_definition_id = "c31620e3-55ff-4af6-ae86-884aa0daa9b2"
cap = Captures(data_root)
captures = cap.filter(def_id=bounding_box_definition_id)
captures.head(3)

### Visualize

In [ ]:
import os
import random

from PIL import Image
from datasetinsights.stats import plot_bboxes
from datasetinsights.datasets.synthetic import read_bounding_box_2d

line_width = 5
resize_scale = 2

def draw_bounding_boxes(filename, annotation):
    filepath = os.path.join(data_root, filename)
    image = Image.open(filepath)
    boxes = read_bounding_box_2d(annotation)
    img_with_boxes = plot_bboxes(image, boxes, box_line_width=line_width)
    print(f"Image: {filename}")
    new_size = (img_with_boxes.width // resize_scale, img_with_boxes.height // resize_scale)
    
    return img_with_boxes.resize(new_size)
    

In [ ]:

# pick an index at random
index = random.randrange(captures.shape[0])

filename = captures.loc[index, "filename"]
annotation = captures.loc[index, "annotation.values"]
draw_bounding_boxes(filename, annotation)